<a href="https://colab.research.google.com/github/alicmu2024/GNSS-Jamming-Detection-and-Classification-using-Machine-Learning-Deep-Learning-and-Computer-Vision/blob/main/Jamming_ML_Models_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **`Let's move on to the Machine Learning `**

In [ ]:
# Downloading The Dataset
%%capture
! wget https://zenodo.org/records/3783969/files/Jamming_Classifier.zip?download=1

In [ ]:
%%capture
%cp /content/Jamming_Classifier.zip?download=1 Jamming_Classifier.zip
! rm -rf /content/Jamming_Classifier.zip?download=1
! unzip ./Jamming_Classifier.zip
! rm -rf /content/energyop.m
! rm -rf /content/Channel
! rm -rf /content/Classifier
! rm -rf /content/Detector_Functions
! rm -rf /content/GNSS_signals
! rm -rf /content/Init
! rm -rf /content/Jammer_signals
! rm -rf /content/Misc
! rm -rf /content/Plotting
! rm -rf /content/Progress_Bar
! rm -rf /content/Sim
! rm -rf /content/Test_statistic
! rm -rf /content/CNN_algorithm.m
! rm -rf /content/Jamming_Classifier.zip
! rm -rf /content/SVM_algorithm.m
! rm -rf /content/SVM_algorithm2.m
! rm -rf /content/Testing_CNR_JSR.mat
! rm -rf /content/Training_CNR_JSR.mat
! rm -rf /content/energyop.m
! rm -rf /content/main.m
! rm -rf /content/main_test_classifiers.m
! rm -rf /content/parfor_progress_percentage.txt
! rm -rf /content/plotConfMat.m
! rm -rf /content/stopIfAccuracyNotImproving.m

In [ ]:
import numpy as np
import torch.nn as nn
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision import transforms as tt
from collections import Counter
from torch.utils.data import Subset

In [ ]:
# Define your dataset directories
train_dir = '/content/Image_training_database'
test_dir = '/content/Image_testing_database'

# Data transforms (normalization & data augmentation)
stats = ((0.7582, 0.7582, 0.7582), (0.4282, 0.4282, 0.4282))
train_tfms = tt.Compose([
    tt.Resize(128),
    tt.ToTensor(),
    tt.Normalize(*stats, inplace=True)
])

valid_tfms = tt.Compose([
    tt.Resize(128),
    tt.ToTensor(),
    tt.Normalize(*stats)
])

# Load the datasets
train_ds = ImageFolder(train_dir, train_tfms)
test_ds = ImageFolder(test_dir, valid_tfms)

# Combine the datasets
combined_ds = train_ds + test_ds

# Total number of images in the combined dataset
total_images = len(combined_ds)
print('Total number of images: ', total_images)

# Get the class labels
class_labels = [combined_ds[i][1] for i in range(total_images)]
class_counts = Counter(class_labels)
print("Class distribution in combined dataset:", class_counts)

# Stratified split
train_indices, temp_indices = train_test_split(
    list(range(total_images)),
    test_size=30000,
    stratify=class_labels,
    random_state=42
)

val_indices, test_indices = train_test_split(
    temp_indices,
    test_size=15000,
    stratify=[class_labels[i] for i in temp_indices],
    random_state=42)

# Create subsets for training, validation, and test datasets
train_ds = Subset(combined_ds, train_indices)
valid_ds = Subset(combined_ds, val_indices)
test_ds = Subset(combined_ds, test_indices)

# Check class balance in the training dataset
train_classes = [combined_ds[i][1] for i in train_indices]
train_class_counts = Counter(train_classes)
print("Training dataset class balance:")
print(train_class_counts)

# Check class balance in the validation dataset
valid_classes = [combined_ds[i][1] for i in val_indices]
valid_class_counts = Counter(valid_classes)
print("\nValidation dataset class balance:")
print(valid_class_counts)

# Check class balance in the test dataset
test_classes = [combined_ds[i][1] for i in test_indices]
test_class_counts = Counter(test_classes)
print("\nTest dataset class balance:")
print(test_class_counts)

# PyTorch data loaders
batch_size = 32
train_dl = DataLoader(train_ds,
                      batch_size,
                      shuffle=True,
                      num_workers=2,
                      pin_memory=True)

valid_dl = DataLoader(valid_ds,
                      batch_size,
                      num_workers=2,
                      pin_memory=True)

test_dl = DataLoader(test_ds,
                     batch_size,
                     num_workers=2,
                     pin_memory=True)

Total number of images:  120000
Class distribution in combined dataset: Counter({0: 20000, 1: 20000, 2: 20000, 3: 20000, 4: 20000, 5: 20000})
Training dataset class balance:
Counter({0: 15000, 3: 15000, 5: 15000, 1: 15000, 4: 15000, 2: 15000})

Validation dataset class balance:
Counter({5: 2500, 1: 2500, 0: 2500, 4: 2500, 2: 2500, 3: 2500})

Test dataset class balance:
Counter({2: 2500, 4: 2500, 0: 2500, 3: 2500, 1: 2500, 5: 2500})


In [ ]:
import torch
from torchvision.models import resnet18
from torchvision.models import ResNet18_Weights

# Define the FeatureExtractor class
class FeatureExtractor(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = resnet18(weights=ResNet18_Weights.DEFAULT)
        self.model = nn.Sequential(*list(self.model.children())[:-1])  # Remove the last layer

    def forward(self, x):
        with torch.no_grad():
            features = self.model(x)
        return features.view(features.size(0), -1)

# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Extract features function
def extract_features(data_loader):
    feature_extractor = FeatureExtractor().to(device)  # Move model to device
    features = []
    labels = []

    for images, target in data_loader:
        images = images.to(device)  # Move images to the same device as the model
        feature = feature_extractor(images)
        features.append(feature.cpu().numpy())  # Move features back to CPU
        labels.append(target.numpy())  # Assuming target is already a numpy array

    return np.concatenate(features), np.concatenate(labels)

In [ ]:
features, labels = extract_features(train_dl)
print(features.shape)
print(labels.shape)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 105MB/s]


(90000, 512)
(90000,)


In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features,
                                                    labels,
                                                    test_size=0.2,
                                                    random_state=42)

In [ ]:
# Train Random Forest
rf_model = RandomForestClassifier(n_estimators=10000)
rf_model.fit(X_train, y_train)

# Evaluate the models
for model in [rf_model]:
    y_pred = model.predict(X_test)
    print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.97      1.00      0.99      2059
           1       0.98      0.99      0.98      1997
           2       1.00      1.00      1.00      1983
           3       0.81      0.94      0.87      1999
           4       0.99      0.97      0.98      1939
           5       0.95      0.78      0.86      2023

    accuracy                           0.95     12000
   macro avg       0.95      0.95      0.95     12000
weighted avg       0.95      0.95      0.95     12000



In [ ]:
# Train SVM
svm_model = SVC(kernel='linear')  # You can choose other kernels as well
svm_model.fit(X_train, y_train)

# Evaluate the models
for model in [svm_model]:
    y_pred = model.predict(X_test)
    print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.97      1.00      0.99      2059
           1       0.98      0.99      0.99      1997
           2       1.00      1.00      1.00      1983
           3       0.80      0.94      0.86      1999
           4       1.00      0.98      0.99      1939
           5       0.95      0.77      0.85      2023

    accuracy                           0.95     12000
   macro avg       0.95      0.95      0.95     12000
weighted avg       0.95      0.95      0.95     12000



In [ ]:
# Train XGBoost
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
xgb_model.fit(X_train, y_train)

# Evaluate the models
for model in [xgb_model]:
    y_pred = model.predict(X_test)
    print(classification_report(y_test, y_pred))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2059
           1       1.00      1.00      1.00      1997
           2       1.00      1.00      1.00      1983
           3       0.85      0.90      0.87      1999
           4       1.00      0.99      1.00      1939
           5       0.90      0.84      0.87      2023

    accuracy                           0.95     12000
   macro avg       0.96      0.96      0.96     12000
weighted avg       0.96      0.95      0.95     12000

